# Intro to OpenMP

## Part 1 : Sequential Sum

In [1]:
%%writefile sum.c

#include <stdio.h>
#include <stdlib.h>

typedef unsigned long ulong;

int main(int argc, char **argv) {

    /* get N from command line */
    if (argc != 2) {
	    printf ("Command usage : %s %s\n",argv[0],"N");
	    return 1;
    }

    ulong N = atol(argv[1]);

    ulong sum = 0;
    for (ulong i = 1; i <= N;i++) {
	    sum += i;
    }

    printf ("sum = %lu\n",sum);
    printf ("N*(N+1)/2 = %lu\n",(N/2)*(N+1));
}

Writing sum.c


In [2]:
!gcc -o sum sum.c

In [3]:
%%time 
! ./sum 100000000

sum = 5000000050000000
N*(N+1)/2 = 5000000050000000
CPU times: user 8.3 ms, sys: 7.55 ms, total: 15.9 ms
Wall time: 555 ms


## Part 2 : OpenMP Sum

In [4]:
%%writefile sum_v1.c

#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

typedef unsigned long ulong;

int main(int argc, char **argv) {

    /* get N and thread_count from command line */
    if (argc < 3) {
        printf ("Command usage : %s %s %s\n",argv[0],"N","thread_count");
        return 1;
    }

    ulong N = atol(argv[1]);
    int thread_count = atoi(argv[2]);
    omp_set_num_threads(thread_count);

    /* start the timer */
    double start_time, end_time;
    start_time = omp_get_wtime();

    ulong sum = 0;
#pragma omp parallel
    {
        ulong thread_sum = 0;
#pragma omp for
        for (ulong i=1;i<=N;i++) {
            thread_sum += i;
	    }
#pragma omp atomic
        sum += thread_sum;
    }   

    /* stop the timer */
    end_time = omp_get_wtime();

    printf ("thread_count = %d, ",thread_count);
    printf ("elapsed time = %g\n",end_time-start_time);
    printf (" N = %lu, ",N);
    printf ("sum = %lu, ",sum);
    printf ("N*(N+1)/2 = %lu\n",(N/2)*(N+1));
}

Writing sum_v1.c


In [7]:
!gcc -o sum_v1 sum_v1.c -Xclang -fopenmp

sum_v1.c:4:10: fatal error: 'omp.h' file not found
#include <omp.h>
         ^~~~~~~
1 error generated.


In [6]:
! ./sum_v1 100000000 1
! ./sum_v1 100000000 2
! ./sum_v1 100000000 4

zsh:1: no such file or directory: ./sum_v1
zsh:1: no such file or directory: ./sum_v1
zsh:1: no such file or directory: ./sum_v1
